In [ ]:
import ipywidgets as widget
import json
from typing import Callable

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
from cartopy.feature import ShapelyFeature

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import logic_iterative as logic
import data_structs as ds

# -------- Map (Plotly) -------- #


def getDummyData(placements: dict[ds.Unit, int]) -> dict[str, list[str]]:
    result = {new_list: [] for new_list in ["unit", "code", "group", "metric"]}
    for unit in placements:
        result["unit"].append(unit.name)
        result["code"].append(unit.code)
        result["group"].append("0")
        result["metric"].append(unit.metric)

    return result


def getCharts(state: logic.State) -> go.Figure:
    with open(f"assets/{state.scale}/geo.json", encoding="utf8") as file:
        geojson = json.load(file)

    colors = px.colors.qualitative.Vivid

    data = getDummyData(state.placements)

    fig = make_subplots(
        rows=1,
        cols=2,
        column_widths=[2, 1],
        specs=[[{"type": "choropleth"}, {"type": "pie"}]],
    )
    fig.add_choropleth(
        geojson=geojson,
        locations=data["code"],
        z=data["group"],
        text=data["metric"],
        zmin=0,
        zmax=len(colors),
        legendgroup="group",
        colorscale=colors,
        row=1,
        col=1,
    )
    fig.update_layout(geo={"scope": "usa"}, height=700)
    fig.add_pie(
        values=data["metric"],
        labels=data["group"],
        textinfo="label+value+percent",
        legendgroup="group",
        marker=dict(colors=colors),
        row=1,
        col=2,
    )
    return fig


def updateFigs(state: logic.State, plots: go.Figure):
    metrics = [u.metric for u in state.placements]
    groups = list(state.placements.values())
    # TODO: distinct choropleth traces?
    plots.update_traces(z=groups, text=metrics, selector=dict(type="choropleth"))
    plots.update_traces(labels=groups, values=metrics, selector=dict(type="pie"))


def drawSolvedPlotly(numGroup: int, metricID: str | int, scale: str | int):
    plots = getCharts(logic.State(numGroup=numGroup, scale=scale, metricID=metricID))

    updateFigs(logic.solve(numGroup=numGroup, scale=scale, metricID=metricID), plots)

    plots.show()


# -------- Map (CartoPy) -------- #

colors = ["#D3D3D3", "#462255", "#313B72", "#62A87C", "#DE4D86", "#FCD0A1"]


def makeBlankMap() -> dict[str, plt.Axes]:
    # Initialize map
    projection = ccrs.LambertConformal()
    fig, _ = plt.subplots(figsize=(20, 15))

    maps = {}

    # Continental United States
    maps["48"] = fig.add_axes([-0.05, -0.05, 1.2, 1.2], projection=projection)
    maps["48"].set_extent([-125, -66.5, 20, 50], ccrs.Geodetic())

    # Hawaii
    maps["HI"] = fig.add_axes([0.25, 0.1, 0.15, 0.15], projection=projection)
    maps["HI"].set_extent([-155, -165, 20, 15], ccrs.Geodetic())

    # Alaska
    maps["AK"] = fig.add_axes([0.1, 0.1, 0.2, 0.2], projection=projection)
    maps["AK"].set_extent([-185, -130, 70, 50], ccrs.Geodetic())

    return maps


def configureLegend(ax: plt.Axes, state: logic.State, colors: list[str]):
    handles = []
    for group in state.groups:
        handles.append(mpatches.Rectangle((0, 0), 1, 1, facecolor=colors[group.index]))
        ax.legend(
            handles,
            (group.index for group in state.groups),
            loc="lower left",
            bbox_to_anchor=(0.025, -0.0),
            fancybox=True,
            frameon=False,
            fontsize=15,
        )

    ax.set_title(state.metricID, fontsize=20)


def initializeMap(state: logic.State) -> tuple[dict[str, plt.Axes], dict[str, ShapelyFeature]]:
    # get blank map
    maps = makeBlankMap()
    configureLegend(maps["48"], state, colors=colors)
    reader = shpreader.Reader(f"assets/{state.scale}/shapefile/geo.shp")

    records = list(reader.records())
    shapes = {
        records[i].attributes["FID"]: ShapelyFeature(x, ccrs.PlateCarree()) for i, x in enumerate(reader.geometries())
    }

    return maps, shapes


def drawOneUnit(code: str, shape: ShapelyFeature, placement: int, maps: dict[str, plt.Axes]):
    if code == "AK" or code.startswith("02"):
        a = maps["AK"]
    elif code == "HI" or code.startswith("15"):
        a = maps["HI"]
    else:
        a = maps["48"]

    a.add_feature(shape, color=colors[placement], linewidth=0)


def drawUnits(state: logic.State, maps: dict[str, plt.Axes], shapes: dict[str, ShapelyFeature]):
    for code, shape in shapes.items():
        drawOneUnit(code, shape, state.placements.get(code, 0), maps)


def drawSolvedCartoPy(numGroup: int, metricID: str | int, scale: str | int):
    (maps, shapes) = initializeMap(logic.State(numGroup=numGroup, scale=scale, metricID=metricID))

    logic.solve(
        numGroup=numGroup,
        scale=scale,
        metricID=metricID,
        callback=lambda code, placement: drawOneUnit(code, shapes[code], placement, maps),
    )


# -------- Interactivity -------- #


scale_widget = widget.Dropdown(options=ds.scales)
metric_widget = widget.Dropdown()


# Define a function that updates the content of metrics based on what we select for scale
def update(*_):
    metric_widget.options = ds.metricNames(str(scale_widget.value))
    metric_widget.value = metric_widget.options[0]


scale_widget.observe(update)


@widget.interact_manual(
    numGroup=widget.IntSlider(min=2, max=5),
    scale=scale_widget,
    metricID=metric_widget,
    func={"CartoPy": drawSolvedCartoPy, "Plotly": drawSolvedPlotly},
)
def drawSolved(numGroup: int, metricID: str | int, scale: str | int, func: Callable[[int, str | int, str | int], None]):
    func(numGroup, metricID, scale)